In [29]:
import psycopg2
from tabulate import tabulate
from neo4j import GraphDatabase


**Setting up Neo4j and PostgreSQL connection**

In [30]:
# Neo4J connection
URI = "neo4j://localhost:7687" #Change the URI of the database
USERNAME = "neo4j"        #Change the username of the database
PASSWORD = "password"  #Change the password of the database

def execute_query(query, params=None):
    with GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD)) as driver:
        with driver.session() as session:
            result = session.run(query, params or {})
            return [record.data() for record in result]
        

In [31]:
# PostgreSQL connection
def get_connection():
    try:
        return psycopg2.connect(
            database="database", #Change the name of the database
            user="postgres",   #Change the user of the database
            password="password",  #Change the password of the database
            host="127.0.0.1", 
            port=5432,
        )
    except:
        return False
conn = get_connection()
pg_cur = conn.cursor()

In [32]:
# Function to print the results in a tabulated format
def print_table(data, columns=None):
    if data:
        headers = columns if columns else "keys"
        print(tabulate(data, headers=headers, tablefmt="grid"))


**Application Queries**

**Query 1: This query gets the shortest path from a hospital to a specialty the user needs and returns said path from Neo4j. This result is then processed in Python and used in the Postgres query to return provider and hospital information**

In [33]:
query = """MATCH p=shortestPath((h1:Hospital {FacilityName: 'HOLT MEDICAL CENTER'})-[:WORKS_AT|SPECIALIZES_IN*]-(t:Taxonomy {Description: 'Physician Assistants & Advanced Practice Nursing Providers/Nurse Practitioner, Acute Care'}))
RETURN p;"""
result = execute_query(query)
# print(result) #Print the neo4j output

def extract_facility_names(data):
   facility_names = []
   for entry in data:
       if 'p' in entry:
           for item in entry['p']:
               if isinstance(item, dict) and 'FacilityName' in item:
                   facility_names.append(item['FacilityName'])
   return facility_names


# Get facility names
facility_names = extract_facility_names(result)
target_facility = facility_names[-1]
print("Facilities in path:", target_facility)
print('\n')

postgres_query1 = f"SELECT DISTINCT h.FacilityName, CONCAT(LOWER(p.FirstName), ' ', LOWER(p.LastName)) AS provider_name, p.MCNAProviderType AS provider_type, p.PrimaryCare, p.Specialist, p.SeesChildren, p.BHIndicator, p.LicensureType FROM hospitals h INNER JOIN providernetwork p ON p.oshpd_id = h.oshpd_id WHERE h.FacilityName = '{target_facility}';"
pg_cur.execute(f'''{postgres_query1}''')
details = pg_cur.fetchall()
print('Provider and Hospital Information for the above speciality:')
print_table(details, [desc[0] for desc in pg_cur.description])



Facilities in path: ANZA COMMUNITY HEALTH


Provider and Hospital Information for the above speciality:
+-----------------------+------------------+------------------------+---------------+--------------+----------------+---------------+-----------------+
| facilityname          | provider_name    | provider_type          | primarycare   | specialist   | seeschildren   | bhindicator   | licensuretype   |
+=======================+==================+========================+===============+==============+================+===============+=================+
| ANZA COMMUNITY HEALTH | barrie abbott    | Physician Assistant    | N             | N            | Both           |               | NPA             |
+-----------------------+------------------+------------------------+---------------+--------------+----------------+---------------+-----------------+
| ANZA COMMUNITY HEALTH | jana webb        | Pediatric Primary Care | Y             | N            | Both           |               | MD

**Query 2: Get the best/worst rated hospital (using average) and use this information in neo4j to find specialties provided (compare best and worst)**

In [34]:
pg_cur.execute("""WITH HospitalRatings AS (SELECT
        h.FacilityName,
        AVG(hr.Risk_adjusted_rate) AS Avg_Risk_Adjusted_Rate
    FROM hospital_ratings hr
    JOIN Hospitals h ON hr.OSHPD_ID = h.OSHPD_ID
    GROUP BY h.FacilityName
)
(
    -- Best-rated hospital (Lowest risk-adjusted rate)
    SELECT FacilityName, Avg_Risk_Adjusted_Rate
    FROM HospitalRatings
    ORDER BY Avg_Risk_Adjusted_Rate ASC
    LIMIT 1
)
UNION ALL
(
    -- Worst-rated hospital (Highest risk-adjusted rate)
    SELECT FacilityName, Avg_Risk_Adjusted_Rate
    FROM HospitalRatings
    ORDER BY Avg_Risk_Adjusted_Rate DESC
    LIMIT 1
);""")
results = pg_cur.fetchall()
print('Display the worst and the best rated hospital with their ratings')
print_table(results, [desc[0] for desc in pg_cur.description])


best_rating_hospital = results[0][0]
worst_rating_hospital=results[1][0]

query1 = """
MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)
WHERE h.FacilityName = $hospital_name
RETURN h.FacilityName AS Hospital, COLLECT(DISTINCT t.Description) AS Specialties;
"""

result1 = execute_query(query1, {"hospital_name": worst_rating_hospital})
result2 = execute_query(query1, {"hospital_name": best_rating_hospital})

print('\n\nDisplay the specialities for the worst and the best rated hospital:')
print_table(result1+result2)

Display the worst and the best rated hospital with their ratings
+----------------------------+--------------------------+
| facilityname               |   avg_risk_adjusted_rate |
+============================+==========================+
| WEST COVINA MEDICAL CENTER |                   0      |
+----------------------------+--------------------------+
| VO MEDICAL CENTER          |                  12.9667 |
+----------------------------+--------------------------+


Display the specialities for the worst and the best rated hospital:
+----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Hospital                   | Specialties                                                                                                                                                                 |
+============================+===================

**Query3: Get the hospital that covers the most insurance (Neo4j) and display all the insurances with the average cost of all categories of aid**

In [35]:
query = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:PART_OF]->(sn:SubNetwork)-[:UNDER]->(mcp:ManagedCarePlan)
WITH h, COLLECT(DISTINCT mcp) AS managedCarePlans
WITH h, managedCarePlans, SIZE(managedCarePlans) AS planCount
RETURN h.FacilityName AS hospitalName, planCount, [mcp IN managedCarePlans | mcp.name] AS managedCarePlanNames
ORDER BY planCount DESC LIMIT 5;"""
result = execute_query(query)
print_table(result)

selected_insurance = input('Enter the name of a health plan from the list above to view the average cost across all aid categories.').strip().replace('.','')

pg_cur.execute(f'''WITH InsuranceVariety AS (
                SELECT Health_Plan_Name, 
                    COUNT(DISTINCT Category_of_Aid) AS CategoryVariety
                FROM insurance_data
                GROUP BY Health_Plan_Name
            ), 
            MostVariedInsurance AS (
                SELECT Health_Plan_Name
                FROM InsuranceVariety
                WHERE CategoryVariety = (SELECT MAX(CategoryVariety) FROM InsuranceVariety)
            )
            SELECT i.Health_Plan_Name AS insuranceplan, 
                AVG((NULLIF(REGEXP_REPLACE(i.Lower_Bound, '[$,]', '', 'g'), '')::NUMERIC + 
                        NULLIF(REGEXP_REPLACE(i.Upper_Bound, '[$,]', '', 'g'), '')::NUMERIC) / 2) AS avgcost
            FROM insurance_data i 
            JOIN MostVariedInsurance mvi ON i.Health_Plan_Name = mvi.Health_Plan_Name
            WHERE lower(i.Health_Plan_Name) = lower('{selected_insurance}')
            GROUP BY i.Health_Plan_Name;''')
results = pg_cur.fetchall()

print('\n')
print_table(results, [desc[0] for desc in pg_cur.description])

+---------------------------------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| hospitalName                                |   planCount | managedCarePlanNames                                                                                                                                                                                                                                           |
+=============================================+=============+================================================================================================================================================================================================================================================================+
| TRUECARE                                 

**Query4: For the insurance data, find the rows where the cost is within the range (10 to 40 dollars) and then try to find the corresponding providers in neo4j**

In [36]:
# Step 1: Fetch health plan names from PostgreSQL
pg_cur.execute("""
    SELECT DISTINCT health_plan_name
    FROM insurance_data
    WHERE REPLACE(REPLACE(Lower_Bound, '$', ''), ',', '')::NUMERIC >= 10
    AND REPLACE(REPLACE(Upper_Bound, '$', ''), ',', '')::NUMERIC <= 40;
""")
health_plan_names = [row[0] for row in pg_cur.fetchall()]

# Step 2: Query Neo4j using execute_query function
if health_plan_names:
    query = """
    MATCH (mcp:ManagedCarePlan)-[:UNDER]-(sn:SubNetwork)<-[:PART_OF]-(p:Provider)
    WHERE mcp.name IN $planNames
    RETURN p.FirstName AS FirstName, p.LastName AS LastName, 
           p.ProviderID AS ProviderID, 
           sn.name AS SubNetwork, 
           mcp.name AS ManagedCarePlan LIMIT 10;
    """

    result = execute_query(query, {"planNames": health_plan_names})

    results = [{"FirstName": record["FirstName"], "LastName": record["LastName"], 
                "ProviderID": record["ProviderID"], "SubNetwork": record["SubNetwork"],
                "ManagedCarePlan": record["ManagedCarePlan"]} for record in result]

    if results:
        print_table(results)
    else:
        print("No matching providers found for the selected health plans.")
else:
    print("No matching health plans found in the specified range.")


+-------------+----------------+--------------+-------------------+---------------------------+
| FirstName   | LastName       |   ProviderID | SubNetwork        | ManagedCarePlan           |
+=============+================+==============+===================+===========================+
| RICHARD     | SWARTZENTRUBER |          472 | Kaiser Permanente | Inland Empire Health Plan |
+-------------+----------------+--------------+-------------------+---------------------------+
| BILL        | CHAO           |          473 | Kaiser Permanente | Inland Empire Health Plan |
+-------------+----------------+--------------+-------------------+---------------------------+
| DARREN      | MURTARI        |          433 | Kaiser Permanente | Inland Empire Health Plan |
+-------------+----------------+--------------+-------------------+---------------------------+
| SOMJOT      | BRAR           |          434 | Kaiser Permanente | Inland Empire Health Plan |
+-------------+----------------+--------

**Neo4J Queries**

**Query 1: Get 10 hospitals and the list of specializations they provide**

In [23]:
query1 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)
RETURN h.FacilityName AS Hospital,
COLLECT(DISTINCT t.Description) AS Specializations,
COLLECT(DISTINCT p.FirstName) AS Providers
LIMIT 10;"""

result = execute_query(query1)
print_table(result)


+-----------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Query 2: Get the hospital and its count with the maximum number of specialties offered**

In [24]:
query2 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)
RETURN h.FacilityName AS Hospital, COUNT(DISTINCT t) AS SpecializationCount
ORDER BY SpecializationCount DESC
LIMIT 1;"""

result = execute_query(query2)
print_table(result)


+-----------------------------------+-----------------------+
| Hospital                          |   SpecializationCount |
+===================================+=======================+
| RONALD REAGAN UCLA MEDICAL CENTER |                    78 |
+-----------------------------------+-----------------------+


**Query 3: Connection between 2 hospitals**

In [25]:
query3 = """MATCH p=shortestPath((h1:Hospital {FacilityName: 'HOLT MEDICAL CENTER'})-[:WORKS_AT|SPECIALIZES_IN*]-(h2:Hospital {FacilityName: 'ANZA COMMUNITY HEALTH'}))
RETURN p;"""

result = execute_query(query3)
print_table(result)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Query 4: Gets the path connecting a hospital to a specialty; can find the closest hospital based on that specialty in relationships**

In [26]:
query4 = """MATCH p=shortestPath((h1:Hospital {FacilityName: 'HOLT MEDICAL CENTER'})-[:WORKS_AT|SPECIALIZES_IN*]-(t:Taxonomy {Description: 'Physician Assistants & Advanced Practice Nursing Providers/Nurse Practitioner, Acute Care'}))
RETURN p;"""

result = execute_query(query4)
print_table(result)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Query 5: Hospital with the most number of doctors**

In [44]:
query5 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)
WITH h, COUNT(p) AS providerCount
RETURN h.FacilityName AS hospitalName, providerCount
ORDER BY providerCount DESC LIMIT 10;"""
result = execute_query(query5)
print_table(result)

+-----------------------------------------------------+-----------------+
| hospitalName                                        |   providerCount |
+=====================================================+=================+
| MAXIM HEALTHCARE SERVICES HOLDINGS INC              |            4039 |
+-----------------------------------------------------+-----------------+
| UCSF MEDICAL CTR                                    |            3153 |
+-----------------------------------------------------+-----------------+
| BEHAVIORAL HEALTH WORKS                             |            2462 |
+-----------------------------------------------------+-----------------+
| LOS ANGELES MEDICAL CENTER-LAL                      |            2037 |
+-----------------------------------------------------+-----------------+
| OC ANAHEIM MEDICAL CENTER-OCAL                      |            1645 |
+-----------------------------------------------------+-----------------+
| RONALD REAGAN UCLA MEDICAL CENTER   

**Query 6: The most connected doctor -> works in multiple locations**

In [45]:
query6 = """MATCH (p:Provider)-[:WORKS_AT]->(h:Hospital)
WITH p.FirstName + ' ' + p.LastName AS providerName,
COLLECT(DISTINCT h) AS hospitals,
COUNT(DISTINCT h) AS hospitalCount
RETURN providerName, hospitalCount,
[hospital IN hospitals | hospital.FacilityName] AS hospitalNames
ORDER BY hospitalCount DESC LIMIT 10;"""
result = execute_query(query6)
print_table(result)

+-------------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| providerName            |   hospitalCount | hospitalNames                                                                                                                                                                                                                       |
+=========================+=================+=====================================================================================================================================================================================================================================+
| FIDEL SANTACRUZ         |               6 | ['LOS ANGELES COMMUNITY HOSPITAL', 'ST. FRANCIS MEDICAL CENTER', 'MONTEREY PARK HOSPITAL', 'ALHAMBRA HOSPITAL MEDICAL CENTER',

**Query 7: Hospital with the most insurances covered; or most covered insurance**

In [46]:
query7 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:PART_OF]->(sn:SubNetwork)-[:UNDER]->(mcp:ManagedCarePlan)
WITH h, COLLECT(DISTINCT mcp) AS managedCarePlans
WITH h, managedCarePlans, SIZE(managedCarePlans) AS planCount
RETURN h.FacilityName AS hospitalName, planCount, [mcp IN managedCarePlans | mcp.name] AS managedCarePlanNames
ORDER BY planCount DESC LIMIT 10;"""
result = execute_query(query7)
print_table(result)

+---------------------------------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| hospitalName                                |   planCount | managedCarePlanNames                                                                                                                                                                                                                                           |
+=============================================+=============+================================================================================================================================================================================================================================================================+
| TRUECARE                                 

**Query 8: Compare doctors based on specialty**

In [47]:
query8 = """MATCH (p1:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)<-[:SPECIALIZES_IN]-(p2:Provider)
WHERE p1.FirstName + " " + p1.LastName < p2.FirstName + " " + p2.LastName
WITH p1.FirstName + " " + p1.LastName AS Doctor1,
     p2.FirstName + " " + p2.LastName AS Doctor2,
     t.Description AS SharedSpecialty
WITH DISTINCT Doctor1, Doctor2, SharedSpecialty
LIMIT 100
RETURN Doctor1, Doctor2, SharedSpecialty
ORDER BY Doctor1, Doctor2 LIMIT 10;"""
result = execute_query(query8)
print_table(result)


+----------------+---------------------------+------------------------------------------------------+
| Doctor1        | Doctor2                   | SharedSpecialty                                      |
+================+===========================+======================================================+
| Azra Zarrinpar | JONATHAN LYNNE            | Allopathic & Osteopathic Physicians/General Practice |
+----------------+---------------------------+------------------------------------------------------+
| Azra Zarrinpar | PETIA KREMEN              | Allopathic & Osteopathic Physicians/General Practice |
+----------------+---------------------------+------------------------------------------------------+
| Azra Zarrinpar | SHAHRZAD SODAGAR-MARVASTI | Allopathic & Osteopathic Physicians/General Practice |
+----------------+---------------------------+------------------------------------------------------+
| Azra Zarrinpar | XIAOGUANG WANG            | Allopathic & Osteopathic Physicians

**PostgreSQL Queries**

**Query 1: Rank the hospitals by the number of primary care physicians they have**

In [48]:
query1 = '''SELECT
            h.FacilityName,
            COUNT(p.Specialist) FILTER (WHERE p.PrimaryCare = 'Y') AS primary_count,
            RANK() OVER (ORDER BY COUNT(p.PrimaryCare) FILTER (WHERE p.PrimaryCare = 'Y') DESC) AS rank
            FROM
            hospitals h
            JOIN
            providernetwork p ON p.OSHPD_ID = h.OSHPD_ID
            GROUP BY
            h.FacilityName LIMIT 10;'''

pg_cur.execute(query1)
results = pg_cur.fetchall()
print('Hospitals ranked by the number of primary care physicians:')
print_table(results, [desc[0] for desc in pg_cur.description])


Hospitals ranked by the number of primary care physicians:
+----------------------------------------+-----------------+--------+
| facilityname                           |   primary_count |   rank |
+========================================+=================+========+
| SAC HEALTH SYSTEM                      |             154 |      1 |
+----------------------------------------+-----------------+--------+
| MAIN CAMPUS COMMUNITY HEALTH CENTER    |              98 |      2 |
+----------------------------------------+-----------------+--------+
| CLINICA DEL VALLE DEL PAJARO           |              57 |      3 |
+----------------------------------------+-----------------+--------+
| SALUD PARA LA GENTE                    |              57 |      3 |
+----------------------------------------+-----------------+--------+
| TRUECARE                               |              45 |      5 |
+----------------------------------------+-----------------+--------+
| LOS ANGELES COMMUNITY HOSPITA

**Query 2: Percentage of Specialists in Each Hospital**

In [49]:
query2 = '''SELECT 
            h.FacilityName,
            COUNT(p.Specialist) FILTER (WHERE p.Specialist = 'Y') AS total_specialists,
            COUNT(p.Specialist) AS total_providers,
            ROUND(100.0 * COUNT(p.Specialist) FILTER (WHERE p.Specialist = 'Y') / COUNT(p.Specialist), 2) AS specialist_percentage
        FROM 
            hospitals h
        JOIN 
            providernetwork p ON p.OSHPD_ID = h.OSHPD_ID
        GROUP BY 
            h.FacilityName LIMIT 10;'''

pg_cur.execute(query2)
results = pg_cur.fetchall()
print('Percentage of Specialists in Each Hospital:')
print_table(results, [desc[0] for desc in pg_cur.description])


Percentage of Specialists in Each Hospital:
+--------------------------------------+---------------------+-------------------+-------------------------+
| facilityname                         |   total_specialists |   total_providers |   specialist_percentage |
+======================================+=====================+===================+=========================+
| COMPREHENSIVE AUTISM CENTER INC      |                   0 |               261 |                    0    |
+--------------------------------------+---------------------+-------------------+-------------------------+
| SHARP GROSSMONT HOSPITAL             |                  12 |                12 |                  100    |
+--------------------------------------+---------------------+-------------------+-------------------------+
| SANTA CRUZ WOMEN'S HEALTH CENTER     |                   3 |                60 |                    5    |
+--------------------------------------+---------------------+-------------------+--

**Query 3: Find the top 5 counties with the most number of providers:**

In [50]:
query3 = '''SELECT h.County, COUNT(p.ProviderID) AS ProviderCount
            FROM ProviderNetwork p
            JOIN Hospitals h ON p.OSHPD_ID = h.OSHPD_ID
            GROUP BY h.County
            ORDER BY ProviderCount DESC
            LIMIT 10;'''

pg_cur.execute(query3)
results = pg_cur.fetchall()
print('Top 5 counties with the most number of providers:')
print_table(results, [desc[0] for desc in pg_cur.description])

Top 5 counties with the most number of providers:
+----------------+-----------------+
| county         |   providercount |
+================+=================+
| San Diego      |            8818 |
+----------------+-----------------+
| Los Angeles    |            7638 |
+----------------+-----------------+
| San Francisco  |            4172 |
+----------------+-----------------+
| Orange         |            1699 |
+----------------+-----------------+
| Riverside      |            1591 |
+----------------+-----------------+
| Santa Cruz     |             457 |
+----------------+-----------------+
| Merced         |             402 |
+----------------+-----------------+
| San Bernardino |             381 |
+----------------+-----------------+
| Monterey       |             348 |
+----------------+-----------------+
| Fresno         |             346 |
+----------------+-----------------+


**Query 4: Comparing each hospital’s bed count with the county’s average:**

In [51]:
query4 = '''SELECT FacilityName, County, Total_number_beds,
                AVG(Total_number_beds) OVER (PARTITION BY County) AS AvgBedsInCounty,
                Total_number_beds - AVG(Total_number_beds) OVER (PARTITION BY County) AS DifferenceFromAvg
            FROM Hospitals
            WHERE Total_number_beds IS NOT NULL LIMIT 10;
            '''

pg_cur.execute(query4)
results = pg_cur.fetchall()
print('Compare hospital\'s bed count with the county\'s average:')
print_table(results, [desc[0] for desc in pg_cur.description])

Compare hospital's bed count with the county's average:
+--------------------------------------------+--------------+---------------------+-------------------+---------------------+
| facilityname                               | county       |   total_number_beds |   avgbedsincounty |   differencefromavg |
+============================================+==============+=====================+===================+=====================+
| UCSF BENIOFF CHILDREN'S HOSPITAL - OAKLAND | Alameda      |                 163 |            163    |                0    |
+--------------------------------------------+--------------+---------------------+-------------------+---------------------+
| COLUSA REGIONAL MEDICAL CENTER CLINICS     | Colusa       |                 167 |            167    |                0    |
+--------------------------------------------+--------------+---------------------+-------------------+---------------------+
| KAISER FOUNDATION HOSPITAL - WALNUT CREEK  | Contra Costa | 

**Query 5: Most common specialisation in Providers**

In [52]:
query5 = '''SELECT t.PROVIDER_TAXONOMY_DESCRIPTION, COUNT(*) AS Provider_Count
            FROM ProviderNetwork p
            JOIN taxonomy_data t ON p.Taxonomy = t.PROVIDER_TAXONOMY_CODE
            GROUP BY t.PROVIDER_TAXONOMY_DESCRIPTION
            ORDER BY Provider_Count DESC LIMIT 10;
            '''

pg_cur.execute(query5)
results = pg_cur.fetchall()
print('Most common specialisation in Providers:')
print_table(results, [desc[0] for desc in pg_cur.description])

Most common specialisation in Providers:
+----------------------------------------------------------------------------------------------------+------------------+
| provider_taxonomy_description                                                                      |   provider_count |
+====================================================================================================+==================+
| Behavioral Health & Social Service Providers/Technician                                            |             6373 |
+----------------------------------------------------------------------------------------------------+------------------+
| Physician Assistants & Advanced Practice Nursing Providers/Physician Assistant                     |             1189 |
+----------------------------------------------------------------------------------------------------+------------------+
| Allopathic & Osteopathic Physicians/Emergency Medicine                                             |   

**Query 6: Calculate the Percentage of Providers Offering Telehealth in Each County**

In [5]:
query6 = '''SELECT h.County,
                ROUND(COUNT(CASE WHEN p.Telehealth IN ('Both', 'Only') THEN 1 END) * 100.0 / COUNT(*), 2) AS Telehealth_Percentage
            FROM ProviderNetwork p
            JOIN Hospitals h ON p.OSHPD_ID = h.OSHPD_ID
            GROUP BY h.County
            ORDER BY Telehealth_Percentage DESC LIMIT 10;;

            '''

pg_cur.execute(query6)
results = pg_cur.fetchall()
print('Percentage of Providers Offering Telehealth in Each County:')
print_table(results, [desc[0] for desc in pg_cur.description])

Percentage of Providers Offering Telehealth in Each County:
+---------------+-------------------------+
| county        |   telehealth_percentage |
+===============+=========================+
| Stanislaus    |                  100    |
+---------------+-------------------------+
| Santa Barbara |                   91.3  |
+---------------+-------------------------+
| Kings         |                   88.24 |
+---------------+-------------------------+
| San Joaquin   |                   70    |
+---------------+-------------------------+
| Sacramento    |                   63.48 |
+---------------+-------------------------+
| San Diego     |                   62.26 |
+---------------+-------------------------+
| Kern          |                   53.57 |
+---------------+-------------------------+
| Placer        |                   50    |
+---------------+-------------------------+
| Tulare        |                   42.74 |
+---------------+-------------------------+
| Contra Costa  

**Query 7: List hospitals with poor ratings and high adverse events**

In [55]:
query7 = '''SELECT h.FacilityName, hr.Performance_measure, hr.No_of_adverse_events, hr.Risk_adjusted_rate, hr.Hospital_ratings
            FROM Hospitals h
            JOIN hospital_ratings hr ON h.OSHPD_ID = hr.OSHPD_ID
            WHERE hr.Hospital_ratings = 'Worse' AND hr.No_of_adverse_events > 50
            ORDER BY hr.Risk_adjusted_rate DESC LIMIT 10;
            '''

pg_cur.execute(query7)
results = pg_cur.fetchall()
print('Hospitals with poor ratings and high adverse events:')
print_table(results, [desc[0] for desc in pg_cur.description])

Hospitals with poor ratings and high adverse events:
+-------------------------------------------------------------+-----------------------+------------------------+----------------------+--------------------+
| facilityname                                                | performance_measure   |   no_of_adverse_events |   risk_adjusted_rate | hospital_ratings   |
+=============================================================+=======================+========================+======================+====================+
| LOS ROBLES HOSPITAL AND MEDICAL CENTER                      | Acute Stroke          |                     61 |                 15   | Worse              |
+-------------------------------------------------------------+-----------------------+------------------------+----------------------+--------------------+
| LOS ANGELES MEDICAL CENTER-LAL                              | Acute Stroke          |                     51 |                 12.2 | Worse              |
+----